# Quantitative Value Investing Strategy

According to Investopedia, "value investing is an investment strategy that involves picking stocks that appear to be trading for less than their intrinsic or book value."

For this project, I will select the top 50 stocks with the best value metrics to recommend trades for an equal-weight portfolio of the 50 stocks.

## Library Imports

Import open-source software libraries.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
from statistics import mean
import xlsxwriter

## Import Stock List and API Token

In [2]:
stocks = pd.read_csv('sp500_constituents.csv')
stocks = stocks.replace('WLTW', 'WTW')
stocks = stocks.replace('VIAC', 'PARA')
from secrets import IEX_CLOUD_API_TOKEN

## API Call

First, I rank securities based on only one metric, which is the price-to-earnings ratio.

In [3]:
symbol = 'FB'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 56084750,
 'calculationPrice': 'close',
 'change': -1.58,
 'changePercent': -0.00771,
 'close': 211.02,
 'closeSource': 'ilfaocfi',
 'closeTime': 1682991318446,
 'companyName': 'Meta Platforms Inc - Class A',
 'currency': 'USD',
 'delayedPrice': 210.026,
 'delayedPriceTime': 1651647979823,
 'extendedChange': -1.58,
 'extendedChangePercent': -0.0076,
 'extendedPrice': 207.3,
 'extendedPriceTime': 1712191245255,
 'high': 216.91,
 'highSource': 'pi neeri51dl amucet yde',
 'highTime': 1657655032226,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 210.09,
 'iexCloseTime': 1681410134698,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 213.147,
 'iexOpenTime': 1710821191141,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1692814258241,
 'latestPrice': 210.99,
 'latestSource': 'Close',
 'latestTime': 'February 18, 2022',
 'latestUpdate': 1646143770027,
 'latestVolum

## Parse API Call

The API call has the metrics needed for this project.

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

7.7

## Batch API Call & Building Our DataFrame

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks(stocks['Symbol'], 100))

symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [6]:
df_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

df = pd.DataFrame(columns = df_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = df_columns
            ),
            ignore_index = True
        )
        
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,152.11,7.6,N/A
1,AOS,74.70,11.84,N/A
2,ABT,121.20,29.08,N/A
3,ABBV,148.24,35.46,N/A
4,ABMD,306.04,105.77,N/A
...,...,...,...,...
500,YUM,131.00,25.42,N/A
501,ZBRA,420.47,13.44,N/A
502,ZBH,123.68,31.9,N/A
503,ZION,71.05,6.79,N/A


## Remove Growth Stocks

According to NerdWallet, "value investors look for stocks they believe are undervalued by the market (value stocks), while growth investors seek stocks that they think will deliver better-than-average returns (growth stocks). Often growth and value are pitted against each other as an either-or option."

Since this project is dedicated to identifying the top 50 value stocks in the S&P 500, I will remove the growth (glamor) stocks from our DataFrame by ranking all stocks based on P/E ratio and remove those outside of top 50.

In [7]:
df.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
df = df[ df['Price-to-Earnings Ratio'] > 0 ][:50]
df.reset_index(drop = True, inplace = True)
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BKNG,2735.72,292.31,N/A
1,KSU,303.24,289.89,N/A
2,FIS,96.37,260.8,N/A
3,NOW,581.58,256.62,N/A
4,HES,96.11,151.34,N/A
5,EQIX,698.90,145.97,N/A
6,DXCM,393.24,127.6,N/A
7,IFF,139.60,115.22,N/A
8,SBAC,291.11,113.95,N/A
9,CRM,199.39,113.04,N/A


## Calculate Number of Shares to Buy

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Please enter the value of your portfolio: ')
    try:
        print(float(portfolio_size))
    except ValueError:
        print('Please enter a number.')
        portfolio_size = input('Please once again enter the value of your portfolio: ')
    
portfolio_input()

Please enter the value of your portfolio: 100000000
100000000.0


In [9]:
position_size = float(portfolio_size) / len(df)

for row in df.index:
    df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / df.loc[row, 'Price'])
    
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BKNG,2735.72,292.31,731
1,KSU,303.24,289.89,6595
2,FIS,96.37,260.8,20753
3,NOW,581.58,256.62,3438
4,HES,96.11,151.34,20809
5,EQIX,698.90,145.97,2861
6,DXCM,393.24,127.6,5085
7,IFF,139.60,115.22,14326
8,SBAC,291.11,113.95,6870
9,CRM,199.39,113.04,10030


## Better Value Strategy

Every valuation metric has its pros and cons. For instance, the P/E ratio used above does not apply well to stocks with negative earnings; P/B ratio may not be able to properly stock buybacks.

Hence, we will use a composite basket of valuation metrics to devise a more robust quantitative value strategies by filtering **for** stocks with the lowest percentiles of the following metrics: P/E ratio, P/B ratio, P/S ratio, EV/EBITDA, and EV/GP.

According to Investopedia,
- The price-to-earnings ratio (P/E ratio) is the ratio for valuing a company that measures its current share price relative to its earnings per share (EPS).
- The P/B ratio measures the market's valuation of a company relative to its book value. The market value of equity is typically higher than the book value of a company, P/B ratio is used by value investors to identify potential investments. P/B ratios under 1 are typically considered solid investments.
- The P/S ratio is an investment valuation ratio that shows a company's market capitalization divided by the company's sales for the previous 12 months. It is a measure of the value investors are receiving from a company's stock by indicating how much are they are paying for the stock per dollar of the company's sales.
- The enterprise value to earnings before interest, taxes, depreciation, and amortization ratio (EV/EBITDA) compares the value of a company—debt included—to the company’s cash earnings less non-cash expenses. EV calculates a company's total value or assessed worth, while EBITDA measures a company's overall financial performance and profitability. Typically, when evaluating a company, an EV/EBITDA value below 10 is seen as healthy. It's best to use the EV/EBITDA metric when comparing companies within the same industry or sector.

And, according to Corporate Finance Institute,
- The EV/Gross Profit Ratio is a profitability financial ratio that estimates the enterprise value of a company to its gross profit. It demonstrates how many dollars of enterprise value are generated for every dollar of gross profit earned. Generally, the lower the ratio, the lower is the company's net worth.

In [10]:
symbol = 'FB'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

pe_ratio = data[symbol]['quote']['peRatio']

pb_ratio = data[symbol]['advanced-stats']['priceToBook']

ps_ratio = data[symbol]['advanced-stats']['priceToSales']

ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = ev/ebitda

gp = data[symbol]['advanced-stats']['grossProfit']
ev_to_gp_ratio = ev/gp


`rv` stands for robust value. According to ThoughtCo.com, robust statistics ... are any statistics that yield good performance when data is drawn from a wide range of probability distributions that are largely unaffected by outliers or small departures from model assumptions in a given dataset. In other words, a robust statistic is resistant to errors in the results.

In [11]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gp = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = ev/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gp = ev/gp
        except TypeError:
            ev_to_gp = np.NaN
            
        rv_df = rv_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A', 
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gp,
                    'N/A',
                    'RV Score'
                ],
                index = rv_columns
            ),
            ignore_index = True
        )
        
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,149.67,N/A,7.6,N/A,2.8,N/A,1.23,N/A,5.966665,N/A,3.202575,N/A,RV Score
1,AOS,74.50,N/A,12.13,N/A,3.23,N/A,1.63,N/A,7.996206,N/A,4.039707,N/A,RV Score
2,ABT,120.70,N/A,28.92,N/A,6.0,N/A,4.92,N/A,20.401657,N/A,8.819483,N/A,RV Score
3,ABBV,148.60,N/A,35.77,N/A,19.71,N/A,4.64,N/A,12.574849,N/A,8.612072,N/A,RV Score
4,ABMD,301.22,N/A,102.81,N/A,9.61,N/A,14.04,N/A,47.118730,N/A,16.208265,N/A,RV Score
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.00,N/A,24.8,N/A,-4.87,N/A,5.86,N/A,20.482012,N/A,9.944723,N/A,RV Score
501,ZBRA,437.50,N/A,14.06,N/A,3.87,N/A,1.99,N/A,10.257229,N/A,4.458652,N/A,RV Score
502,ZBH,126.08,N/A,31.41,N/A,2.01,N/A,3.34,N/A,17.976827,N/A,5.784668,N/A,RV Score
503,ZION,73.14,N/A,6.78,N/A,1.57,N/A,3.3,N/A,5.482685,N/A,2.903238,N/A,RV Score


## Missing Data

As a result of invalid EV/EBITDA and/or EV/GP.

There are two main ways to deal with missing data:
- Drop missing data from the data set (pd's `dropna`)
- Replace missing data with a certain value (pd's `fillna`), which is the method that I will use in this project. I will fill in the blanks with the average of available data from the column.

In [12]:
rv_df[ rv_df.isnull().any(axis=1) ]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
24,GOOG,2734.17,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
43,AON,286.60,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
64,BRK.B,320.60,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
149,DISCK,30.88,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
191,FRT,122.23,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
205,FOX,38.82,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
332,NWS,22.97,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
428,TROW,147.85,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
459,UA,14.62,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
473,PARA,28.81,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score


In [13]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_df[column].fillna(rv_df[column].mean(), inplace = True)
    
rv_df[ rv_df.isnull().any(axis=1) ]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


That there is no `Nan` left is as we expect.

## Calculate Value Percentiles

In [14]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_df.index:
    for metric in metrics.keys():
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric], rv_df.loc[row, metric])
        
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,149.67,N/A,7.60,15.643564,2.80,49.70297,1.23,23.762376,5.966665,14.455446,3.202575,22.970297,RV Score
1,AOS,74.50,N/A,12.13,31.683168,3.23,55.148515,1.63,33.267327,7.996206,27.326733,4.039707,30.49505,RV Score
2,ABT,120.70,N/A,28.92,70.09901,6.00,76.930693,4.92,74.653465,20.401657,77.623762,8.819483,70.891089,RV Score
3,ABBV,148.60,N/A,35.77,81.287129,19.71,94.653465,4.64,72.475248,12.574849,50.49505,8.612072,68.910891,RV Score
4,ABMD,301.22,N/A,102.81,97.425743,9.61,85.148515,14.04,96.435644,47.118730,97.227723,16.208265,91.485149,RV Score
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.00,N/A,24.80,64.158416,-4.87,4.950495,5.86,81.188119,20.482012,77.821782,9.944723,76.039604,RV Score
501,ZBRA,437.50,N/A,14.06,36.336634,3.87,62.970297,1.99,40.39604,10.257229,41.980198,4.458652,34.455446,RV Score
502,ZBH,126.08,N/A,31.41,75.643564,2.01,37.029703,3.34,57.623762,17.976827,69.70297,5.784668,47.722772,RV Score
503,ZION,73.14,N/A,6.78,13.663366,1.57,25.544554,3.30,57.029703,5.482685,12.475248,2.903238,20.19802,RV Score


## Calculate RV Score

Our simple `rv` will be the arithmetic mean of the four percentiles.

In [15]:
for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,149.67,N/A,7.60,15.643564,2.80,49.70297,1.23,23.762376,5.966665,14.455446,3.202575,22.970297,25.306931
1,AOS,74.50,N/A,12.13,31.683168,3.23,55.148515,1.63,33.267327,7.996206,27.326733,4.039707,30.49505,35.584158
2,ABT,120.70,N/A,28.92,70.09901,6.00,76.930693,4.92,74.653465,20.401657,77.623762,8.819483,70.891089,74.039604
3,ABBV,148.60,N/A,35.77,81.287129,19.71,94.653465,4.64,72.475248,12.574849,50.49505,8.612072,68.910891,73.564356
4,ABMD,301.22,N/A,102.81,97.425743,9.61,85.148515,14.04,96.435644,47.118730,97.227723,16.208265,91.485149,93.544554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.00,N/A,24.80,64.158416,-4.87,4.950495,5.86,81.188119,20.482012,77.821782,9.944723,76.039604,60.831683
501,ZBRA,437.50,N/A,14.06,36.336634,3.87,62.970297,1.99,40.39604,10.257229,41.980198,4.458652,34.455446,43.227723
502,ZBH,126.08,N/A,31.41,75.643564,2.01,37.029703,3.34,57.623762,17.976827,69.70297,5.784668,47.722772,57.544554
503,ZION,73.14,N/A,6.78,13.663366,1.57,25.544554,3.30,57.029703,5.482685,12.475248,2.903238,20.19802,25.782178


## Top 50 Value Stocks

In [16]:
rv_df.sort_values('RV Score', ascending = False, inplace = True)
rv_df = rv_df[:50].reset_index(drop = True)

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NVDA,246.94,N/A,75.47,95.841584,25.68,96.039604,25.38,99.60396,61.847280,97.821782,37.453001,99.80198,97.821782
1,MPWR,462.16,N/A,107.39,97.821782,18.53,93.861386,19.88,99.009901,79.429118,99.009901,31.974802,99.405941,97.821782
2,FTNT,311.03,N/A,92.26,96.831683,44.74,98.217822,16.73,97.821782,73.107084,98.811881,20.607173,96.435644,97.623762
3,TECH,418.30,N/A,81.26,96.435644,9.71,85.346535,15.80,97.623762,42.592540,96.435644,21.800539,97.623762,94.693069
4,TYL,426.54,N/A,109.18,98.019802,7.77,82.376238,11.84,94.851485,81.845390,99.207921,27.852856,98.811881,94.653465
5,TSLA,858.37,N/A,90.56,96.633663,14.72,91.683168,8.00,88.910891,45.163152,96.633663,30.651551,99.207921,94.613861
6,MKTX,390.27,N/A,52.04,90.49505,13.80,90.39604,20.86,99.405941,34.632768,94.455446,20.202259,95.841584,94.118812
7,BKNG,2621.97,N/A,282.71,99.80198,19.61,94.455446,11.88,95.049505,54.677573,97.623762,11.678025,83.564356,94.09901
8,INTU,483.10,N/A,63.51,95.049505,13.89,90.693069,12.85,96.039604,47.123707,97.425743,15.300099,90.693069,93.980198
9,ABMD,301.22,N/A,102.81,97.425743,9.61,85.148515,14.04,96.435644,47.118730,97.227723,16.208265,91.485149,93.544554


## Calculate Number of Shares to Buy

In [17]:
portfolio_input()

Please enter the value of your portfolio: 100000000
100000000.0


In [18]:
position_size = float(portfolio_size) / len(rv_df.index)
for i in range(0, len(rv_df['Ticker'])-1):
    rv_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_df['Price'][i])
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NVDA,246.94,8099,75.47,95.841584,25.68,96.039604,25.38,99.60396,61.847280,97.821782,37.453001,99.80198,97.821782
1,MPWR,462.16,4327,107.39,97.821782,18.53,93.861386,19.88,99.009901,79.429118,99.009901,31.974802,99.405941,97.821782
2,FTNT,311.03,6430,92.26,96.831683,44.74,98.217822,16.73,97.821782,73.107084,98.811881,20.607173,96.435644,97.623762
3,TECH,418.30,4781,81.26,96.435644,9.71,85.346535,15.80,97.623762,42.592540,96.435644,21.800539,97.623762,94.693069
4,TYL,426.54,4688,109.18,98.019802,7.77,82.376238,11.84,94.851485,81.845390,99.207921,27.852856,98.811881,94.653465
5,TSLA,858.37,2329,90.56,96.633663,14.72,91.683168,8.00,88.910891,45.163152,96.633663,30.651551,99.207921,94.613861
6,MKTX,390.27,5124,52.04,90.49505,13.80,90.39604,20.86,99.405941,34.632768,94.455446,20.202259,95.841584,94.118812
7,BKNG,2621.97,762,282.71,99.80198,19.61,94.455446,11.88,95.049505,54.677573,97.623762,11.678025,83.564356,94.09901
8,INTU,483.10,4139,63.51,95.049505,13.89,90.693069,12.85,96.039604,47.123707,97.425743,15.300099,90.693069,93.980198
9,ABMD,301.22,6639,102.81,97.425743,9.61,85.148515,14.04,96.435644,47.118730,97.227723,16.208265,91.485149,93.544554


## Format Excel Output

In [19]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name = 'Value Strategy', index = False)

In [22]:
font_color = '#000000'
background_color = '#FFFFFF'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

float_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format':'0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [23]:
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Number of Shares to Buy', integer_format],
                    'D': ['Price-to-Earnings Ratio', float_format],
                    'E': ['PE Percentile', percent_format],
                    'F': ['Price-to-Book Ratio', float_format],
                    'G': ['PB Percentile',percent_format],
                    'H': ['Price-to-Sales Ratio', float_format],
                    'I': ['PS Percentile', percent_format],
                    'J': ['EV/EBITDA', float_format],
                    'K': ['EV/EBITDA Percentile', percent_format],
                    'L': ['EV/GP', float_format],
                    'M': ['EV/GP Percentile', percent_format],
                    'N': ['RV Score', percent_format]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()